In [1]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
N_W = 32
N_H = 32
N_C = 3
N_CLASS = 369
bool = 'test.tfrecords'     # 选择测试集或者训练集  train/test
pics_dir = './data/hasy-data'  #图片目录
tfrecords_dir = './data/tfrecords'  #tfrecords目录


In [3]:
#获取图片路径和标签
def data_label():
    filelist = []
    for file in glob.glob(r'data/classification-task/*/test.csv'):    #train/test
        df = pd.read_csv(file)
        filelist.append(df)
    df = pd.concat(filelist)
    paths = list(df['path'])
    label_list = list(df['symbol_id'])

    df.drop_duplicates(['symbol_id'], inplace=True)
    print(df)
    old_name = list(df['latex'])
    old_label = list(df['symbol_id'])
    label_dict = dict(map(reversed, enumerate(old_label)))
    print(label_dict)
    return paths, label_list, label_dict

In [4]:
# 制作tfrecords数据
def maketfrecord(paths, label_list, label_dict, bool):  

    writer = tf.python_io.TFRecordWriter(os.path.join(tfrecords_dir, bool))
    for i in tqdm(range(len(paths))):
        index = label_dict.get(label_list[i])
        path= paths[i].split('/')[-1]
        # print(label_list[i], index, path)

        img = Image.open(os.path.join(pics_dir, path))
        img = img.resize((N_W, N_H))
        img_raw = img.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))

        writer.write(example.SerializeToString())

    writer.close()
    print("生成成功")

In [ ]:
paths, label_list, label_dict = data_label()
    # print(paths)
maketfrecord(paths, label_list, label_dict, bool)